# Deploying Models with Flask and FastAPI

## 📚 Learning Objectives

By completing this notebook, you will:
- Deploy machine learning models using Flask
- Deploy models using FastAPI
- Compare Flask vs FastAPI for ML deployment
- Create RESTful APIs for model serving
- Handle model predictions via HTTP requests

## 🔗 Prerequisites

- ✅ Understanding of basic deep learning models
- ✅ Python 3.8+ installed
- ✅ Basic understanding of web APIs

---

## Official Structure Reference

This notebook covers practical activities from **Course 08, Unit 5**:
- Deploying models using Flask, FastAPI, and TensorFlow Serving
- **Source:** `DETAILED_UNIT_DESCRIPTIONS.md` - Unit 5 Practical Content

---

## Introduction

**Model Deployment** is the process of making trained models available for use in production environments. Flask and FastAPI are popular Python web frameworks for creating APIs that serve ML models.


In [1]:
# Try importing required libraries
try:
    import flask
    HAS_FLASK = True
    print(f"✅ Flask {flask.__version__} available")
except ImportError:
    HAS_FLASK = False
    print("⚠️  Flask not available. Install with: pip install flask")

try:
    import fastapi
    HAS_FASTAPI = True
    print(f"✅ FastAPI available")
except ImportError:
    HAS_FASTAPI = False
    print("⚠️  FastAPI not available. Install with: pip install fastapi uvicorn")

try:
    import sklearn
    import pickle
    HAS_SKLEARN = True
    print("✅ scikit-learn available")
except ImportError:
    HAS_SKLEARN = False
    print("⚠️  scikit-learn not available")

import numpy as np
print("✅ NumPy ready!")


/var/folders/7n/l2c2z2x57871xg4f_0drsv1m0000gn/T/ipykernel_73374/703698168.py:5: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(f"✅ Flask {flask.__version__} available")


✅ Flask 3.1.0 available
✅ FastAPI available


✅ scikit-learn available
✅ NumPy ready!


## Part 1: Simple Model for Deployment

First, let's create and train a simple model to deploy.


In [2]:
if HAS_SKLEARN:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    
    # Create a simple model
    X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Save model
    with open('deployment_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    print("=" * 60)
    print("Model Created and Saved")
    print("=" * 60)
    print(f"Training accuracy: {model.score(X_train, y_train):.4f}")
    print(f"Test accuracy: {model.score(X_test, y_test):.4f}")
    print("✅ Model saved as 'deployment_model.pkl'")
else:
    print("Note: Install scikit-learn to create a model for deployment")


Model Created and Saved
Training accuracy: 1.0000
Test accuracy: 0.8800
✅ Model saved as 'deployment_model.pkl'


## Part 2: Flask Deployment

Flask is a lightweight web framework perfect for simple ML model APIs.


In [3]:
if HAS_FLASK and HAS_SKLEARN:
    # Flask app code (would be in a separate .py file in production)
    flask_app_code = '''
from flask import Flask, request, jsonify
import pickle
import numpy as np

app = Flask(__name__)

# Load model
with open('deployment_model.pkl', 'rb') as f:
    model = pickle.load(f)

@app.route('/predict', methods=['POST'])
def predict():
    """Predict endpoint"""
    try:
        data = request.json
        features = np.array(data['features']).reshape(1, -1)
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0].tolist()
        
        return jsonify({
            'prediction': int(prediction), 'probabilities': probability,
            'status': 'success'
        })
    except Exception as e:
        return jsonify({'error': str(e), 'status': 'error'}), 400

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
'''
    
    # Save Flask app
    with open('flask_app.py', 'w') as f:
        f.write(flask_app_code)
    
    print("=" * 60)
    print("Flask Deployment Code Created")
    print("=" * 60)
    print("✅ Flask app saved as 'flask_app.py'")
    print("\nTo run Flask app:")
    print("  python flask_app.py")
    print("\nExample API call:")
    print("  curl -X POST http://localhost:5000/predict \\")
    print("       -H 'Content-Type: application/json' \\")
    print("       -d '{\"features\": [0.5, -0.3, 0.1, 0.8, -0.2, 0.4, 0.9, -0.1, 0.3, 0.6]}'")
else:
    print("=" * 60)
    print("Flask Deployment (Installation Required)")
    print("=" * 60)
    print("""
    To deploy with Flask:
    
    1. Install Flask:
       pip install flask
    
    2. Create app.py:
       from flask import Flask, request, jsonify
       import pickle
       
       app = Flask(__name__)
       model = pickle.load(open('model.pkl', 'rb'))
       
       @app.route('/predict', methods=['POST'])
       def predict():
           data = request.json
           prediction = model.predict([data['features']])
           return jsonify({'prediction': int(prediction[0])})
       
       if __name__ == '__main__':
           app.run(debug=True)
    
    3. Run:
       python app.py
    """)


Flask Deployment Code Created
✅ Flask app saved as 'flask_app.py'

To run Flask app:
  python flask_app.py

Example API call:
  curl -X POST http://localhost:5000/predict \
       -H 'Content-Type: application/json' \
       -d '{"features": [0.5, -0.3, 0.1, 0.8, -0.2, 0.4, 0.9, -0.1, 0.3, 0.6]}'


## Part 3: FastAPI Deployment

FastAPI is a modern, fast web framework with automatic API documentation.


In [4]:
if HAS_FASTAPI and HAS_SKLEARN:
    # FastAPI app code (would be in a separate .py file in production)
    fastapi_app_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pickle
import numpy as np

app = FastAPI(title="ML Model API", version="1.0.0")

# Load model
with open('deployment_model.pkl', 'rb') as f:
    model = pickle.load(f)

class PredictionRequest(BaseModel):
    features: list[float]

class PredictionResponse(BaseModel):
    prediction: int
    probabilities: list[float]
    status: str

@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    """Predict endpoint with automatic API documentation"""
    try:
        features = np.array(request.features).reshape(1, -1)
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0].tolist()
        
        return PredictionResponse(
            prediction=int(prediction), probabilities=probability,
            status='success'
        )
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/health")
async def health():
    """Health check endpoint"""
    return {"status": "healthy"}

# Run with: uvicorn fastapi_app:app --reload
'''
    
    # Save FastAPI app
    with open('fastapi_app.py', 'w') as f:
        f.write(fastapi_app_code)
    
    print("=" * 60)
    print("FastAPI Deployment Code Created")
    print("=" * 60)
    print("✅ FastAPI app saved as 'fastapi_app.py'")
    print("\nTo run FastAPI app:")
    print("  uvicorn fastapi_app:app --reload")
    print("\nAPI Documentation available at:")
    print("  http://localhost:8000/docs")
    print("\nExample API call:")
    print("  curl -X POST http://localhost:8000/predict \\")
    print("       -H 'Content-Type: application/json' \\")
    print("       -d '{\"features\": [0.5, -0.3, 0.1, 0.8, -0.2, 0.4, 0.9, -0.1, 0.3, 0.6]}'")
else:
    print("=" * 60)
    print("FastAPI Deployment (Installation Required)")
    print("=" * 60)
    print("""
    To deploy with FastAPI:
    
    1. Install FastAPI and Uvicorn:
       pip install fastapi uvicorn
    
    2. Create app.py:
       from fastapi import FastAPI
       from pydantic import BaseModel
       import pickle
       
       app = FastAPI()
       model = pickle.load(open('model.pkl', 'rb'))
       
       class Request(BaseModel):
           features: list[float]
       
       @app.post("/predict")
       async def predict(request: Request):
           prediction = model.predict([request.features])
           return {"prediction": int(prediction[0])}
    
    3. Run:
       uvicorn app:app --reload
    
    4. View API docs:
       http://localhost:8000/docs
    """)


FastAPI Deployment Code Created
✅ FastAPI app saved as 'fastapi_app.py'

To run FastAPI app:
  uvicorn fastapi_app:app --reload

API Documentation available at:
  http://localhost:8000/docs

Example API call:
  curl -X POST http://localhost:8000/predict \
       -H 'Content-Type: application/json' \
       -d '{"features": [0.5, -0.3, 0.1, 0.8, -0.2, 0.4, 0.9, -0.1, 0.3, 0.6]}'


## Part 4: Flask vs FastAPI Comparison


In [5]:
print("=" * 60)
print("Flask vs FastAPI Comparison")
print("=" * 60)

comparison = {
    "Performance": {
        "Flask": "Good, synchronous by default",
        "FastAPI": "Excellent, async support, faster"
    },
    "API Documentation": {
        "Flask": "Manual (need Swagger/Flask-RESTX)",
        "FastAPI": "Automatic (OpenAPI/Swagger built-in)"
    },
    "Type Validation": {
        "Flask": "Manual validation needed",
        "FastAPI": "Automatic with Pydantic"
    },
    "Learning Curve": {
        "Flask": "Easier for beginners",
        "FastAPI": "Slightly steeper, but more powerful"
    },
    "Use Case": {
        "Flask": "Simple APIs, traditional web apps",
        "FastAPI": "Modern APIs, high performance, async workloads"
    }
}

for aspect, details in comparison.items():
    print(f"\n{aspect}:")
    print(f"  Flask: {details['Flask']}")
    print(f"  FastAPI: {details['FastAPI']}")

print("\n✅ Recommendation:")
print("  - Use Flask for simple, traditional deployments")
print("  - Use FastAPI for modern, high-performance APIs with automatic docs")


Flask vs FastAPI Comparison

Performance:
  Flask: Good, synchronous by default
  FastAPI: Excellent, async support, faster

API Documentation:
  Flask: Manual (need Swagger/Flask-RESTX)
  FastAPI: Automatic (OpenAPI/Swagger built-in)

Type Validation:
  Flask: Manual validation needed
  FastAPI: Automatic with Pydantic

Learning Curve:
  Flask: Easier for beginners
  FastAPI: Slightly steeper, but more powerful

Use Case:
  Flask: Simple APIs, traditional web apps
  FastAPI: Modern APIs, high performance, async workloads

✅ Recommendation:
  - Use Flask for simple, traditional deployments
  - Use FastAPI for modern, high-performance APIs with automatic docs


## Summary

### Key Concepts:
1. **Flask**: Lightweight, simple, great for basic ML model APIs
2. **FastAPI**: Modern, fast, automatic API docs, async support
3. **RESTful API**: Standard way to serve models via HTTP requests
4. **Model Serialization**: Using pickle or joblib to save/load models

### Deployment Steps:
1. Train and save model (pickle/joblib)
2. Create web app (Flask/FastAPI)
3. Load model in app
4. Create predict endpoint
5. Deploy to server (local, cloud, or container)

### Best Practices:
- Add health check endpoints
- Validate input data
- Handle errors gracefully
- Use environment variables for configuration
- Add logging and monitoring
- Consider authentication for production

**Reference:** Course 08, Unit 5: "Deploying models using Flask, FastAPI, and TensorFlow Serving"
